# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

[In this notebook you're provided with hints and brief instructions and thinking prompts. Don't ignore them as they are designed to equip you with the structure for the project and will help you analyze what you're doing on a deeper level. Before submitting your project, make sure you remove all hints and descriptions provided to you. Instead, make this report look as if you're sending it to your teammates to demonstrate your findings - they shouldn't know you had some external help from us! To help you out, we've placed the hints you should remove in square brackets.]

[Before you dive into analyzing your data, explain the purposes of the project and hypotheses you're going to test.]
check the relationship between marital status and loan payments
check the financial situation and the payment of the loan
check the relationship between employment and loan repayment
the higher the financial situation, the greater the chances of timely repayment of the loan
married people with children are more likely to repay a loan
people who have a job are more likely to pay off a loan




In [4]:
import pandas as pd
import numpy as np

# Loading all the libraries
try:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv')
    
except:
    data = pd.read_csv('C:/Users/Admin/Desktop/practicum/modul_1/Proj2/credit_scoring_eng.csv')


print(list(data.columns.values))# Load the data


['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan



In [5]:
f'the data has {len(data)} rows'
f'the data has {len(data.columns)} columns'# Let's see how many rows and columns our dataset has



'the data has 12 columns'

In [6]:
data.head(10)# let's print the first N rows



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


*days_employed column has many negative values
*education column has uppercase and lowercase letters
*total_income and days_employed columns has missing values


In [7]:
data.info()# Get info on data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


 there are missing values in two columns. the number is the same, we need to compare these columns to get a better understanding of the database.

The number of missing values in both columns is the same. Maybe people who don't work don't get paid?


In [10]:
missing_data = data[(data['days_employed'].isnull())&(data['total_income'].isnull())]# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

missing_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


the number of rows in the filtered table match the number of missing values. clients who do not have the number of working days also do not have a monthly income. maybe they don't work.

calculate percentage of missing date


In [11]:

len(data) / data.isna().sum()


children                 inf
days_employed       9.901104
dob_years                inf
education                inf
education_id             inf
family_status            inf
family_status_id         inf
gender                   inf
income_type              inf
debt                     inf
total_income        9.901104
purpose                  inf
dtype: float64

In [12]:
# Checking distribution

missing_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB



**Possible reasons for missing values in data**
1. technical reason -  if so, then we have nothing to do with it
2. it's possible that people who don't have a job really don't have an income.
3. there may be other reasons, but in order to find them, we need to perform some actions with the database.



check each column individually and see how much of each value is missing in the missing information

In [13]:
missing_data[missing_data.days_employed.isnull()]['gender'].value_counts()

F    1484
M     690
Name: gender, dtype: int64

most missing values are wemen.

In [14]:
missing_data[missing_data.days_employed.isnull()]['education'].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

most missing values with secondary education, possibly education influences the final result

In [15]:
missing_data['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

most of the missing values don't have children, maybe the number of children affects the final result

In [16]:
missing_data[missing_data.days_employed.isnull()]['purpose'].value_counts()

having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
purchase of the house for my family         71
transactions with my real estate            71
transactions with commercial real estate    70
housing renovation                          70
buy commercial real estate                  67
buying property for renting out             65
property                                    62
real estate transactions                    61
buy residential real estate                 61
housing                                     60
building a property                         59
cars                                        57
going to university                         56
to become educated                          55
second-hand car purchase                    54
buying my own

there are too many meanings and perhaps after categorization it will be seen better, but it is already clear that most of the meanings refer to the wedding, perhaps there is a pattern here

In [17]:
missing_data[missing_data.days_employed.isnull()]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

again, most of the missing values are related to employees, it is possible that there is a pattern here too

we see that the greatest matches with the missing dates are among women, those who do not have children and secondary education. the rest is not suspicious. perhaps on follow-up we will see the relationship between gender, education and the number of children in debt repayment. 

In [18]:
missing_data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
F,1484,0,1484,1484,1484,1484,1484,1484,1484,0,1484
M,690,0,690,690,690,690,690,690,690,0,690


In [19]:
# Check for other reasons and patterns that could lead to missing values
missing_data.groupby('education_id').count()

,children,days_employed,dob_years,education,family_status,family_status_id,gender,income_type,debt,total_income,purpose
education_id,,,,,,,,,,,
0,544,0,544,544,544,544,544,544,544,0,544
1,1540,0,1540,1540,1540,1540,1540,1540,1540,0,1540
2,69,0,69,69,69,69,69,69,69,0,69
3,21,0,21,21,21,21,21,21,21,0,21


In [20]:
data['education'].unique()# Let's see all values in education column to check if and what spellings will need to be fixed


array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [21]:
data['education'] = data['education'].str.lower()# Fix the registers if required


In [22]:
# Checking all the values in the column to make sure we fixed them
data['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

[Check the data the `children` column]

In [23]:
# Let's see the distribution of values in the `children` column
#групбай
data.groupby('children').size()


children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
dtype: int64

47 lines with a value of -1 and this does not affect and can be replaced or thrown out. 76 lines with a value of 20. in theory, it also does not really affect much and you can just get rid of it or replace it.


In [24]:
# [fix the data based on your decision]
data.loc[data['children'] == -1,'children' ] = 1


In [25]:
data = data.loc[data['children'] != 20]

In [26]:
# Checking the `children` column again to make sure it's all fixed

data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [27]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
data['days_employed'].describe()

count     19284.000000
mean      63141.233527
std      140910.281638
min      -18388.949901
25%       -2747.876441
50%       -1204.164714
75%        -290.206221
max      401755.400475
Name: days_employed, dtype: float64

we see a large number of negative values and numbers with a comma, and the days are whole. perhaps when filling mixed up a hyphen and a minus sign. and also there was a technical error due to which the number became not an integer. also the maximum value of 401755 days is clearly impossible. we need to find out how many such extreme data and if they are a little discard them.

In [28]:
# Address the problematic values, if they exist
data['days_employed'] = data['days_employed'].fillna(0)
data['days_employed'] = data['days_employed'].astype(np.int64).abs()
 


In [29]:
# Check the result - make sure it's fixed
data['days_employed'].describe()

count     21449.000000
mean      60245.349760
std      133440.134589
min           0.000000
25%         612.000000
50%        1811.000000
75%        4791.000000
max      401755.000000
Name: days_employed, dtype: float64

we changed the negative to positive and the float to inter. it remains to understand what to do with outliers (40 thousand +), but there is a huge amount of missing data, therefore, we will not use this column in further calculations. and leave everything as it is. in principle, we can calculate the number of working days more than 1 and less than 365 days multiplied by 20 working years. but no need)))

In [30]:
# Check the `dob_years` for suspicious values and count the percentage

data['dob_years'].describe()

count    21449.000000
mean        43.298615
std         12.576314
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [31]:
data.groupby('dob_years').count()

,children,days_employed,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
dob_years,,,,,,,,,,,
0,100,100,100,100,100,100,100,100,100,90,100
19,14,14,14,14,14,14,14,14,14,13,14
20,51,51,51,51,51,51,51,51,51,46,51
21,110,110,110,110,110,110,110,110,110,92,110
22,183,183,183,183,183,183,183,183,183,166,183
23,253,253,253,253,253,253,253,253,253,217,253
24,263,263,263,263,263,263,263,263,263,242,263
25,356,356,356,356,356,356,356,356,356,334,356
26,407,407,407,407,407,407,407,407,407,372,407


number of problematic data with age = 0 101. not much. so you can easily get rid of them as outliers, but we will replace them with the median value. 

In [32]:
# Address the issues in the `dob_years` column, if they exist
age_median = data['dob_years'].median()
print(age_median)


42.0


In [33]:
data['dob_years'] = data['dob_years'].replace(0,42)

In [34]:
# Check the result - make sure it's fixed
data['dob_years'].describe()

count    21449.000000
mean        43.494429
std         12.222610
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [35]:
# Let's see the values for the column

data['family_status'].describe()

count       21449
unique          5
top       married
freq        12331
Name: family_status, dtype: object

In [36]:
data['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

everything is ok with this column and nothing needs to be done yet

In [37]:
data['gender'].describe()# Let's see the values in the column

count     21449
unique        3
top           F
freq      14189
Name: gender, dtype: object

In [38]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

incomprehensible gender (XNA) check how many there are

In [39]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


only one. can be thrown out without problems

In [40]:
data = data.drop(index=[10701])# Address the problematic values, if they exist

In [41]:
# Check the result - make sure it's fixed
data[data['gender'] == 'XNA']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [42]:
data['income_type'].describe() # Let's see the values in the column

count        21448
unique           8
top       employee
freq         11076
Name: income_type, dtype: object

In [43]:
data['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [44]:
data['income_type'].value_counts()

employee                       11076
business                        5062
retiree                         3847
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


we have strange values:
1st unemployed - 2
2. entrepreneur - 2
3. paternity / maternity leave -1
4. student - 1
what do we do? unemployed, maternity leave and students are removed. entrepreneurs are replaced by businesses.

In [45]:
data['income_type'] = data['income_type'].replace('entrepreneur', 'business')# Address the problematic values, if they exist

In [46]:
data['income_type'].value_counts()

employee                       11076
business                        5064
retiree                         3847
civil servant                   1457
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [47]:
data.drop(data.index[data['income_type'] == 'unemployed'], inplace=True)

In [48]:
data.drop(data.index[data['income_type'] == 'paternity / maternity leave'], inplace=True)

In [49]:
data.drop(data.index[data['income_type'] == 'student'], inplace=True)

In [50]:
data['income_type'].value_counts()

employee         11076
business          5064
retiree           3847
civil servant     1457
Name: income_type, dtype: int64

In [51]:
# Checking duplicates
data.duplicated().sum()



72

In [52]:
data[data.duplicated()]# Address the duplicates, if they exist

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,0,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,0,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,0,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,0,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,0,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,0,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,0,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [53]:
data = data.drop_duplicates()


In [54]:
data.duplicated().sum()# Last check whether we have any duplicates


0

In [55]:
data.info()# Check the size of the dataset that you now have after your first manipulations with it

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21372 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21372 non-null  int64  
 1   days_employed     21372 non-null  int64  
 2   dob_years         21372 non-null  int64  
 3   education         21372 non-null  object 
 4   education_id      21372 non-null  int64  
 5   family_status     21372 non-null  object 
 6   family_status_id  21372 non-null  int64  
 7   gender            21372 non-null  object 
 8   income_type       21372 non-null  object 
 9   debt              21372 non-null  int64  
 10  total_income      19279 non-null  float64
 11  purpose           21372 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


there were 21525 rows, we cleaned up and merged and it became 21465. in the days_employed column, we replaced the missing date with 0.

# Working with missing values

In [56]:
# Find the dictionaries

we can define dictionaries, for example: education_id and education, but I prefer to use the transform method in the subsequent analysis

### Restoring missing values in `total_income`

In [57]:
data['dob_years'].describe()

count    21372.000000
mean        43.475576
std         12.217662
min         19.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [234]:
# Let's write a function that calculates the age category
def age_group(age):
    if age >= 19 and age < 33:
        return 'young'
    elif age >= 33 and age < 42:
        return 'young+'
    elif age >= 42 and age < 65:
        return 'adults'
    elif age >= 65 and age <= 75:
        return 'pensioner'
    
    

In [235]:
age_group(45) # Test if the function works


'adults'

In [236]:
data['age_group'] = data['dob_years'].apply(age_group)# Creating new column based on function



In [237]:
data['age_group'].value_counts()# Checking how values in the new column



adults       10465
young+        5247
young         4766
pensioner      894
Name: age_group, dtype: int64

factors affecting income:
1. education
2. way of earning
3. age


In [238]:

new_data = data.dropna(subset=['total_income'])
new_data.head(10)# Create a table without missing values and print a few of its rows to make sure it looks fine

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adults
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,young+
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young+
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adults
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adults
7,0,152,50,secondary education,1,married,0,M,employee,0,21731.829,education,adults
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,young+
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,young+


In [239]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19279 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19279 non-null  int64  
 1   days_employed     19279 non-null  int64  
 2   dob_years         19279 non-null  int64  
 3   education         19279 non-null  object 
 4   education_id      19279 non-null  int64  
 5   family_status     19279 non-null  object 
 6   family_status_id  19279 non-null  int64  
 7   gender            19279 non-null  object 
 8   income_type       19279 non-null  object 
 9   debt              19279 non-null  int64  
 10  total_income      19279 non-null  float64
 11  purpose           19279 non-null  object 
 12  age_group         19279 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


In [240]:
new_data['total_income'].mean()# Look at the mean values for income based on your identified factors

26788.659822708647

In [241]:
new_data['total_income'].median()# Look at the median values for income based on your identified factors


23203.786

In [242]:
new_data['total_income'].describe()

count     19279.000000
mean      26788.659823
std       16487.538349
min        3306.762000
25%       16488.504500
50%       23203.786000
75%       32541.860500
max      362496.645000
Name: total_income, dtype: float64

most of the data is in between.16488 and 32541 we will take the median  for analysis based on the columns that we think affect the final result ('age_group','education','income_type')
we take the median and not the average, because we see that there is a very large variation in extreme (especially in very high) wage levels. between 3000 and 360000. there may be values that greatly affect the average

In [ ]:
data['total_income'] = data['total_income'].fillna(data\
.groupby(['age_group','education','income_type'])['total_income'].transform(lambda grp: grp.fillna(np.median(grp))))#  Write a function that we will use for filling in missing values
     

In [ ]:
data['total_income'].isna().sum() # Check if it works


In [ ]:
# Checking the number of entries in the columns

data.info()

###  Restoring values in `days_employed`

In [ ]:
# Distribution of `days_employed` medians based on your identified parameters

data['days_employed'].median()


In [247]:
# Distribution of `days_employed` means based on your identified parameters
data['days_employed'].mean()

60427.82374134381

In [248]:
data['days_employed'].describe()

count     21372.000000
mean      60427.823741
std      133599.546074
min           0.000000
25%         623.000000
50%        1823.000000
75%        4803.000000
max      401755.000000
Name: days_employed, dtype: float64

we have replaced the missing values with 0 at the beginning. because of this, the average value is strongly changed in the negative direction. so we can't use it and will use the median

In [249]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
data['days_employed'] = data['days_employed'].fillna(data\
.groupby(['age_group','education','income_type'])['days_employed'].transform(lambda grp: grp.fillna(np.median(grp))))

In [250]:
# Check that the function works

data['days_employed'].isna().sum() 

0

In [251]:
data.info()# Check the entries in all columns - make sure we fixed all missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21372 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21372 non-null  int64  
 1   days_employed     21372 non-null  int64  
 2   dob_years         21372 non-null  int64  
 3   education         21372 non-null  object 
 4   education_id      21372 non-null  int64  
 5   family_status     21372 non-null  object 
 6   family_status_id  21372 non-null  int64  
 7   gender            21372 non-null  object 
 8   income_type       21372 non-null  object 
 9   debt              21372 non-null  int64  
 10  total_income      19279 non-null  float64
 11  purpose           21372 non-null  object 
 12  age_group         21372 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.3+ MB


## Categorization of data




In [252]:
# Print the values for your selected data for categorization

data['purpose']

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21520       housing transactions
21521          purchase of a car
21522                   property
21523          buying my own car
21524               to buy a car
Name: purpose, Length: 21372, dtype: object

In [253]:
# Check the unique values
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [254]:
wedding = ['to have a wedding', 'having a wedding', 'wedding ceremony' ]
real_estate = ['purchase of the house', 'housing transactions', 'purchase of the house for my family', 'buy real estate', 'buy commercial real estate', 'buy residential real estate', 'construction of own property', 'property', 'building a property', 'transactions with commercial real estate', 'building a real estate',  'housing', 'transactions with my real estate', 'purchase of my own house', 'real estate transactions',  'buying property for renting out', 'housing renovation' ]
car = ['car purchase', 'buying a second-hand car', 'buying my own car', 'cars', 'second-hand car purchase', 'car',  'to own a car', 'purchase of a car', 'to buy a car' ]
education = ['supplementary education', 'education', 'to become educated', 'getting an education', 'to get a supplementary education', 'getting higher education', 'profile education', 'university education', 'going to university' ]


In [255]:
# Let's write a function to categorize the data based on common topics
def purpose_cat(purpose):
    if purpose in real_estate:
        return 'real estate'
    elif purpose in wedding:
        return 'wedding'
    elif purpose in car:
        return 'car'
    elif purpose in education:
        return 'education'
    else:
        return 'other'

In [256]:
purpose_cat('to buy a car')

'car'

In [257]:
# Create a column with the categories and count the values for them

data['purpose_cati'] = data['purpose'].apply(purpose_cat)

In [258]:
data['purpose_cati'].value_counts()

real estate    10771
car             4289
education       3998
wedding         2314
Name: purpose_cati, dtype: int64

In [259]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_cati
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adults,real estate
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,young+,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young+,real estate
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adults,wedding


In [260]:
data.describe()# Looking through all the numerical data in your selected column for categorization


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21372.000000,21372.000000,21372.000000,21372.000000,21372.000000,21372.000000,19279.000000
mean,0.475154,60427.823741,43.475576,0.817004,0.974406,0.080994,26788.659823
std,0.751951,133599.546074,12.217662,0.549003,1.421787,0.272832,16487.538349
min,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,623.000000,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,1823.000000,42.000000,1.000000,0.000000,0.000000,23203.786000
75%,1.000000,4803.000000,53.000000,1.000000,1.000000,0.000000,32541.860500
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,362496.645000


In [261]:
# Getting summary statistics for the column

data['total_income'].describe()

count     19279.000000
mean      26788.659823
std       16487.538349
min        3306.762000
25%       16488.504500
50%       23203.786000
75%       32541.860500
max      362496.645000
Name: total_income, dtype: float64

In [262]:
# Creating function for categorizing into different numerical groups based on ranges
def total_income_cut(total):
    if total <= 17203:
        return 'low'
    elif total <= 24010:
        return 'middle'
    elif total <= 32079:
        return 'high'
    else:
        return 'very high'


In [263]:
total_income_cut(19567)

'middle'

In [264]:
# Creating column with categories
data['income_cat'] = data['total_income'].apply(total_income_cut)

In [265]:
# Count each categories values to see the distribution
data['income_cat'].value_counts()

very high    7104
low          5341
middle       4789
high         4138
Name: income_cat, dtype: int64

In [266]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_cati,income_cat
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adults,real estate,very high
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,young+,car,middle
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,young+,real estate,middle
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,young,education,very high
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adults,wedding,high


## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [267]:
# Check the children data and paying back on time
children_pivot = pd.pivot_table(data, index='children', columns='debt', values='income_cat', aggfunc='count', margins=True)
children_pivot['defauolt_rate'] = (children_pivot[1]/children_pivot['All'])*100
children_pivot
# Calculating default-rate based on the number of children



debt,0,1,All,defauolt_rate
children,,,,
0,13024.0,1063.0,14087,7.545964
1,4410.0,444.0,4854,9.147095
2,1858.0,193.0,2051,9.410044
3,303.0,27.0,330,8.181818
4,37.0,4.0,41,9.756098
5,9.0,NaN,9,NaN
All,19641.0,1731.0,21372,8.099382


the distribution is:
0 children - 7.54 percent do not paid on time
1 - 9.15
2-9.41
3 - 8.18
4 - 9.76

Based on this, we can argue that people without children pay better than those who have children.  peaple with 2 and 4 kids are the worst payers, but there is  only 41 peaple with 4 kids, so it could n't be a representative. the result for people with 5 children does not fit this theory, but there are too few data (only 9) to make an analysis. I think the most dangerous clients are those with 2 kids. (9.41%)

**Is there a correlation between family status and paying back on time?**

In [268]:
# Check the family status data and paying back on time

family_pivot = pd.pivot_table(data, index='family_status', columns='debt', values='income_cat', aggfunc='count', margins=True)
family_pivot['defauolt_rate'] = (family_pivot[1]/family_pivot['All'])*100
family_pivot

# Calculating default-rate based on family status



debt,0,1,All,defauolt_rate
family_status,,,,
civil partnership,3751,385,4136,9.308511
divorced,1109,84,1193,7.041073
married,11362,926,12288,7.535807
unmarried,2527,273,2800,9.750000
widow / widower,892,63,955,6.596859
All,19641,1731,21372,8.099382


**Conclusion**

[Write your conclusions based on your manipulations and observations.]

based on the distribution, the most dangerous clients are not married or living in a civil marriage (9.750000 and 9.308511)

**Is there a correlation between income level and paying back on time?**

In [269]:
# Check the income level data and paying back on time

income_pivot = pd.pivot_table(data, index='income_cat', columns='debt', values='total_income', aggfunc='count', margins=True)
income_pivot['defauolt_rate'] = (income_pivot[1]/income_pivot['All'])*100
income_pivot

# Calculating default-rate based on income level



debt,0,1,All,defauolt_rate
income_cat,,,,
high,3768,370,4138,8.941518
low,4916,425,5341,7.957311
middle,4375,414,4789,8.644811
very high,4659,352,5011,7.024546
All,17718,1561,19279,8.096893


**Conclusion**

[Write your conclusions based on your manipulations and observations.]

judging by the findings, the worst payers are those who earns 'high' and 'middle' - (8.742044 and 8.871421)  


**How does credit purpose affect the default rate?**

In [270]:
# Check the percentages for default rate for each credit purpose and analyze them
purpose_pivot = pd.pivot_table(data, index='purpose_cati', columns='debt', values='income_cat', aggfunc='count', margins=True)
purpose_pivot['defauolt_rate'] = (purpose_pivot[1]/purpose_pivot['All'])*100
purpose_pivot


debt,0,1,All,defauolt_rate
purpose_cati,,,,
car,3889,400,4289,9.326183
education,3629,369,3998,9.229615
real estate,9992,779,10771,7.232383
wedding,2131,183,2314,7.908384
All,19641,1731,21372,8.099382


**Conclusion**

[Write your conclusions based on your manipulations and observations.]


according to the conclusion
the worst payers are those who want a real estate loan - 7.23 and a wedding - 7.87.


# General Conclusion 

[List your conclusions in this final section. Make sure you include all your important conclusions you made that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, and possible reasons and solutions for problematic artifacts that you had to address.]

[List your conclusions regarding the posed questions here as well.]


1. The first step was to find and compare the missing values. we found the same amount in the two columns total_income and days_employed. compared them and realized that they are in the same lines. then we checked the distribution for the remaining columns and came to the conclusion that patterns are possible in the columns: 'children', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'total_income', 'purpose'.
2. second stage - we worked with each column separately. found the problematic data and fixed it.
3. operation and replacement of missing data. in the total_income column, we calculated and categorized the data by age and found the average for each group and replaced the missing data with it.
4. categorized the data in the prupose column
5. conducted a comparative analysis using pivot tables and came to the conclusion that:
the worst clients, those who will face problems in repaying the debt, are:
1. clients without children
2. divorced
3. earning the highest salaries
4. wedding or real estate reasons for loan. 